In [29]:
import json
import pandas as pd
import numpy as np

import spotipy

from sklearn.metrics import pairwise_distances
from my_token import *

In [30]:
scope = 'user-read-currently-playing'
# scope = 'user-modify-playback-state'
# scope = 'user-read-recently-played'

# To connect succesfully you need to provide your own Spotify Credentials
# You can do this signing up in https://developer.spotify.com/ and creating a new app.
token = spotipy.util.prompt_for_user_token(
    USERNAME, scope, client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET, redirect_uri=SPOTIPY_REDIRECT_URI)

sp = spotipy.Spotify(auth=token)

### Get features of skipped songs

In [23]:
df_skipped = pd.read_csv('../data/player_data/skipped.csv', usecols=[i for i in range(11)])
df_skipped.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.625,0.726,1,-2.144,1,0.0310,0.335000,0.000000,0.2730,0.559,119.994
1,0.502,0.972,10,-3.960,0,0.0793,0.000068,0.000702,0.0627,0.729,138.019
2,0.518,0.436,4,-14.134,1,0.0349,0.015700,0.000621,0.0790,0.270,80.880
3,0.688,0.835,8,-4.162,1,0.0911,0.058300,0.000003,0.0795,0.880,84.858


In [24]:
df_skipped.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [25]:
df_skipped.mean()

danceability          0.583250
energy                0.742250
key                   5.750000
loudness             -6.100000
mode                  0.750000
speechiness           0.059075
acousticness          0.102267
instrumentalness      0.000331
liveness              0.123550
valence               0.609500
tempo               105.937750
dtype: float64

In [28]:
normalized_d = (df_skipped - df_skipped.mean()) / (df_skipped.max() - df_skipped.min())
normalized_d

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.224462,-0.030317,-0.527778,0.329942,0.25,-0.467138,0.694866,-0.472176,0.710651,-0.082787,0.246001
1,-0.436828,0.428638,0.472222,0.178482,-0.75,0.336522,-0.305134,0.527824,-0.289349,0.195902,0.561460
2,-0.350806,-0.571362,-0.194444,-0.670058,0.25,-0.402246,-0.258461,0.412439,-0.211840,-0.556557,-0.438540
3,0.563172,0.173041,0.250000,0.161635,0.25,0.532862,-0.131271,-0.468088,-0.209463,0.443443,-0.368921


In [39]:
def calc_dist(feature1, feature2):
    dss = np.vstack((feature1, feature2))
    distance_matrix = pairwise_distances(dss)
    return distance_matrix[0][1]

### Get features of the song to be calculated distance

In [31]:
def get_features(track_id):
    features_results = sp.audio_features([track_id])
    json_features = json.dumps(features_results)
    features_data = json.loads(json_features)

    # Convert features dictionary to a list
    features_list = list(features_data[0].values())

    return features_list

In [41]:
song_to_check_id = "3c8iiZGfEammKJuWTErE5x"
feature1 = get_features(song_to_check_id)
feature1 = feature1[:11]
feature1

[0.778, 0.566, 2, -6.959, 0, 0.0311, 0.672, 0.000197, 0.0698, 0.479, 117.971]

In [44]:
distances = []
for i in range(len(normalized_d)):
    feature2 = normalized_d.iloc[i,:]
    dist_tmp = calc_dist(feature1, feature2)
    distances.append(dist_tmp)
    print(dist_tmp)

117.98566191727076
117.65147964558749
118.61783625127826
118.57302116704994


In [46]:
dist_arr = np.array(distances)

In [51]:
dist_arr.mean()

118.20699974529661